# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [357]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import pandas_datareader.data as pdr
import yfinance as yf
import matplotlib
from scipy import stats
import yahoo_fin.stock_info as si

yf.pdr_override()

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [358]:
stocks=pd.read_csv("constituents.csv")
for i in range(0,len(stocks["Symbol"])):
    if "." in stocks["Symbol"][i]:
        stocks["Symbol"][i]=stocks["Symbol"][i].replace(".","-")
stocks

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott,Health Care
3,ABBV,AbbVie,Health Care
4,ACN,Accenture,Information Technology
...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary
499,ZBRA,Zebra Technologies,Information Technology
500,ZBH,Zimmer Biomet,Health Care
501,ZION,Zions Bancorporation,Financials


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [359]:
symbol="AAPL"
data=pdr.get_quote_yahoo(symbol)
# data["priceEpsCurrentYear"]
data.columns

Index(['language', 'region', 'quoteType', 'typeDisp', 'quoteSourceName',
       'triggerable', 'customPriceAlertConfidence', 'currency',
       'averageDailyVolume3Month', 'exchange', 'shortName', 'longName',
       'messageBoardId', 'exchangeTimezoneName', 'exchangeTimezoneShortName',
       'gmtOffSetMilliseconds', 'market', 'esgPopulated', 'marketState',
       'regularMarketChangePercent', 'regularMarketPrice',
       'firstTradeDateMilliseconds', 'priceHint', 'preMarketChange',
       'preMarketChangePercent', 'preMarketTime', 'preMarketPrice',
       'regularMarketChange', 'regularMarketTime', 'regularMarketDayHigh',
       'regularMarketDayRange', 'regularMarketDayLow', 'regularMarketVolume',
       'regularMarketPreviousClose', 'bid', 'ask', 'bidSize', 'askSize',
       'fullExchangeName', 'financialCurrency', 'regularMarketOpen',
       'averageDailyVolume10Day', 'fiftyTwoWeekLowChange',
       'fiftyTwoWeekLowChangePercent', 'fiftyTwoWeekRange',
       'fiftyTwoWeekHighChange

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [360]:
price=data["price"]
pe_ratio=data["priceEpsCurrentYear"]
pe_ratio

AAPL    22.032415
Name: priceEpsCurrentYear, dtype: float64

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [361]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Symbol'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [362]:
final_df=pd.DataFrame(columns=my_columns)
for symbol_group in symbol_groups:
    data=pdr.get_quote_yahoo(symbol_group)[["price","priceEpsCurrentYear"]]
    data["nan"]=np.nan
    data.reset_index(inplace=True)
    data=data.set_axis(my_columns,axis=1)
    final_df=pd.concat([final_df,data])
final_df.reset_index(inplace=True,drop=True)


In [363]:
final_df_tmp=final_df
final_df_tmp

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,MMM,126.6,12.423945,NaN
1,AOS,60.03,19.49026,NaN
2,ABT,113.39,21.722223,NaN
3,ABBV,152.83,11.04263,NaN
4,ACN,284.88,24.836966,NaN
...,...,...,...,...
498,YUM,130.33,29.221973,NaN
499,ZBRA,293.58,16.91129,NaN
500,ZBH,123.01,18.01025,NaN
501,ZION,50.86,9.049823,NaN


In [364]:
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,MMM,126.6,12.423945,NaN
1,AOS,60.03,19.49026,NaN
2,ABT,113.39,21.722223,NaN
3,ABBV,152.83,11.04263,NaN
4,ACN,284.88,24.836966,NaN
...,...,...,...,...
498,YUM,130.33,29.221973,NaN
499,ZBRA,293.58,16.91129,NaN
500,ZBH,123.01,18.01025,NaN
501,ZION,50.86,9.049823,NaN


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [365]:
final_df.sort_values("Price-to-Earnings Ratio",ascending=False, inplace=True)
final_df=final_df[final_df["Price-to-Earnings Ratio"]>0]
final_df=final_df[:50]
final_df.reset_index(inplace=True,drop=True)

In [366]:
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,WDC,37.32,746.39996,NaN
1,AAL,17.08,244.0,NaN
2,UDR,40.47,175.95653,NaN
3,WELL,71.71,170.7381,NaN
4,DXCM,109.51,136.8875,NaN
5,LYV,76.12,89.55294,NaN
6,ILMN,206.44,88.22223,NaN
7,EQIX,718.0,83.00578,NaN
8,CDAY,67.68,78.69767,NaN
9,DLR,106.85,76.87051,NaN


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [367]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [368]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [369]:
position_size=float(portfolio_size)/len(final_df.index)
# position_size
for row in final_df.index:
    final_df.loc[row,"Number of Shares to Buy"]=math.floor(position_size/final_df.loc[row,"Price"])
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,WDC,37.32,746.39996,5359
1,AAL,17.08,244.0,11709
2,UDR,40.47,175.95653,4941
3,WELL,71.71,170.7381,2789
4,DXCM,109.51,136.8875,1826
5,LYV,76.12,89.55294,2627
6,ILMN,206.44,88.22223,968
7,EQIX,718.0,83.00578,278
8,CDAY,67.68,78.69767,2955
9,DLR,106.85,76.87051,1871


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [370]:
symbol="TSLA"
dataval=si.get_stats_valuation(symbol)
# dataval2.set_index(0,inplace=True)
dataval

,0,1
0,Market Cap (intraday),386.51B
1,Enterprise Value,371.28B
2,Trailing P/E,37.86
3,Forward P/E,24.51
4,PEG Ratio (5 yr expected),1.22
5,Price/Sales (ttm),5.64
6,Price/Book (mrq),9.70
7,Enterprise Value/Revenue,4.96
8,Enterprise Value/EBITDA,22.91


In [371]:
data=pd.DataFrame(columns=["EV/EBITDA","PB","PS"])
data["EV/EBITDA"]=pd.Series(dataval.loc[8,1])
data["PB"]=dataval.loc[6,1]
data["PS"]=dataval.loc[5,1]
data["EV"]=dataval.loc[1,1]
data

,EV/EBITDA,PB,PS,EV
0,22.91,9.70,5.64,371.28B


In [372]:
symbol="MSFT"
dataval2=si.get_stats(symbol)
# dataval2.set_index("Attribute",inplace=True)
dataval2

,Attribute,Value
0,Avg Vol (3 month) 3,30.29M
1,Avg Vol (10 day) 3,32.1M
2,Shares Outstanding 5,7.45B
3,Implied Shares Outstanding 6,NaN
4,Float 8,7.45B
5,% Held by Insiders 1,0.06%
6,% Held by Institutions 1,73.35%
7,"Shares Short (Dec 29, 2022) 4",38.19M
8,"Short Ratio (Dec 29, 2022) 4",1.32
9,"Short % of Float (Dec 29, 2022) 4",0.51%


In [373]:
#EBITDA 
dataval2.loc[39,"Value"]
#GP
strtoint(dataval2.loc[38,"Value"])

77140000000

In [374]:
rv_columns=[
    "Ticker",
    "Price",
    "Number of Shares to Buy",
    "Price-to-Earnings Ratio",
    "PE Percentile",
    "Price-to-Book Ratio",
    "PB Percentile",
    "Price-to-Sales Ratio",
    "PS Percentile",
    "EV/EBITDA",
    "EV/EBITDA Percentile",
    "EV/GP",
    "EV/GP Percentile",
    "RV Score"
]
rv_df=pd.DataFrame(columns=rv_columns)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [375]:
import warnings
warnings.simplefilter('ignore', FutureWarning)
m = {'K': 3, 'M': 6, 'B': 9, 'T': 12,"k":3}
def strtoint(x):
        val=0
        try:
                val=float(x)
        except:
                try: 
                        val=int(float(x[:-1]) * 10 ** m[x[-1]] )
                except:
                        return np.nan


        return val
strtoint("30.29M")

30290000

In [377]:
rv_df=pd.DataFrame(columns=["price","priceEpsCurrentYear",'priceToBook',"EV/EBITDA","PB","PS","EV","EV/GP"])

In [378]:
import time
def data_collect(k,j):
        last_time=time.time()
        global rv_df
        for i,symbol in enumerate(stocks["Symbol"][k:j]):
                try:
                        data=pdr.get_quote_yahoo(symbol)[["price","priceEpsCurrentYear",'priceToBook']]
                except:
                        try: 
                                data=pdr.get_quote_yahoo(symbol)[["price","epsForward",'priceToBook']]
                        except:
                                data=pdr.get_quote_yahoo(symbol)[["price","trailingPE",'priceToBook']]
                # data.reindex(["price","nan0","priceEpsCurrentYear","nan1",'priceToBook',"nan2"],axis=1) 
                dataval=si.get_stats_valuation(symbol)
        
                data["EV/EBITDA"]=strtoint(dataval.loc[8,1])
                data["PB"]=float(dataval.loc[6,1])
                data["PS"]=float(dataval.loc[5,1])
                data["EV"]=strtoint(dataval.loc[1,1])  
                dataval2=si.get_stats(symbol)
                data["EV/GP"]=data["EV"]/strtoint(dataval2.loc[38,"Value"])
                rv_df=pd.concat([rv_df,data])
                tdif=time.time()-last_time
                print("Finished",i,symbol,tdif)
                # if tdif>7:
                #         return
                last_time=time.time()
                # time.sleep(2)
    

In [379]:
# takes too long
# data_collect(0,504)
# rv_df.to_csv("003_data.csv")
rv_df=pd.read_csv("003_data.csv",index_col=0)
rv_df

,price,priceEpsCurrentYear,priceToBook,EV/EBITDA,PB,PS,EV,EV/GP,epsForward,trailingPE
MMM,126.60,12.423945,4.969968,8.71,5.08,2.14,8.451000e+10,5.097105,NaN,NaN
AOS,60.03,19.490260,5.247378,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABT,113.39,21.722223,5.541762,15.50,5.55,4.47,2.044200e+11,8.173531,NaN,NaN
ABBV,152.83,11.042630,16.898497,13.01,16.98,4.72,3.293700e+11,8.471451,NaN,NaN
ACN,284.88,24.836966,7.804932,16.97,7.73,2.90,1.750700e+11,8.886802,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
YUM,130.33,29.221973,-4.333211,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZBRA,293.58,16.911290,5.725262,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZBH,123.01,18.010250,2.107635,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZION,50.86,9.049823,1.787886,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [380]:
rv_df.isnull().any()

price                  False
priceEpsCurrentYear     True
priceToBook            False
EV/EBITDA               True
PB                      True
PS                      True
EV                      True
EV/GP                   True
epsForward              True
trailingPE              True
dtype: bool

In [381]:

for i in rv_df.index:
    if np.isnan(rv_df.loc[i,"priceEpsCurrentYear"]):
        if not(np.isnan(rv_df.loc[i,"epsForward"])):
            rv_df.loc[i,"priceEpsCurrentYear"]=rv_df.loc[i,"price"]/rv_df.loc[i,"epsForward"]
        if not(np.isnan(rv_df.loc[i,"trailingPE"])):
            rv_df.loc[i,"priceEpsCurrentYear"]=rv_df.loc[i,"price"]/rv_df.loc[i,"trailingPE"]
rv_df.isnull().any()

price                  False
priceEpsCurrentYear    False
priceToBook            False
EV/EBITDA               True
PB                      True
PS                      True
EV                      True
EV/GP                   True
epsForward              True
trailingPE              True
dtype: bool

In [382]:
rv_df.drop(["PB","EV"],axis=1,inplace=True)
rv_df.drop(["epsForward","trailingPE"],axis=1,inplace=True)
rv_df.columns

Index(['price', 'priceEpsCurrentYear', 'priceToBook', 'EV/EBITDA', 'PS',
       'EV/GP'],
      dtype='object')

In [383]:
rv_df[  ["Number of Shares to Buy",
    "PE Percentile",
    "PB Percentile",
    "PS Percentile",
    "EV/EBITDA Percentile",
    "EV/GP Percentile",
    "RV Score"]]=np.nan
rv_df=rv_df.reindex([
'index', 'price',  'Number of Shares to Buy', 'priceEpsCurrentYear', 'PE Percentile','priceToBook',
       'PB Percentile', 'PS', 'PS Percentile',  'EV/EBITDA',
        'EV/EBITDA Percentile','EV/GP',
       'EV/GP Percentile', 'RV Score'],axis=1
)
rv_df.set_axis(rv_columns,axis=1,inplace=True)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
MMM,NaN,126.60,NaN,12.423945,NaN,4.969968,NaN,2.14,NaN,8.71,NaN,5.097105,NaN,NaN
AOS,NaN,60.03,NaN,19.490260,NaN,5.247378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABT,NaN,113.39,NaN,21.722223,NaN,5.541762,NaN,4.47,NaN,15.50,NaN,8.173531,NaN,NaN
ABBV,NaN,152.83,NaN,11.042630,NaN,16.898497,NaN,4.72,NaN,13.01,NaN,8.471451,NaN,NaN
ACN,NaN,284.88,NaN,24.836966,NaN,7.804932,NaN,2.90,NaN,16.97,NaN,8.886802,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,NaN,130.33,NaN,29.221973,NaN,-4.333211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZBRA,NaN,293.58,NaN,16.911290,NaN,5.725262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZBH,NaN,123.01,NaN,18.010250,NaN,2.107635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZION,NaN,50.86,NaN,9.049823,NaN,1.787886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [384]:
# rv_df.to_csv("final.csv")

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [385]:
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
MMM,NaN,126.60,NaN,12.423945,NaN,4.969968,NaN,2.14,NaN,8.71,NaN,5.097105,NaN,NaN
AOS,NaN,60.03,NaN,19.490260,NaN,5.247378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABT,NaN,113.39,NaN,21.722223,NaN,5.541762,NaN,4.47,NaN,15.50,NaN,8.173531,NaN,NaN
ABBV,NaN,152.83,NaN,11.042630,NaN,16.898497,NaN,4.72,NaN,13.01,NaN,8.471451,NaN,NaN
ACN,NaN,284.88,NaN,24.836966,NaN,7.804932,NaN,2.90,NaN,16.97,NaN,8.886802,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,NaN,130.33,NaN,29.221973,NaN,-4.333211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZBRA,NaN,293.58,NaN,16.911290,NaN,5.725262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZBH,NaN,123.01,NaN,18.010250,NaN,2.107635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZION,NaN,50.86,NaN,9.049823,NaN,1.787886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [386]:
# rv_df=pd.read_csv("final.csv",index_col=0)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
MMM,NaN,126.60,NaN,12.423945,NaN,4.969968,NaN,2.14,NaN,8.71,NaN,5.097105,NaN,NaN
AOS,NaN,60.03,NaN,19.490260,NaN,5.247378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABT,NaN,113.39,NaN,21.722223,NaN,5.541762,NaN,4.47,NaN,15.50,NaN,8.173531,NaN,NaN
ABBV,NaN,152.83,NaN,11.042630,NaN,16.898497,NaN,4.72,NaN,13.01,NaN,8.471451,NaN,NaN
ACN,NaN,284.88,NaN,24.836966,NaN,7.804932,NaN,2.90,NaN,16.97,NaN,8.886802,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,NaN,130.33,NaN,29.221973,NaN,-4.333211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZBRA,NaN,293.58,NaN,16.911290,NaN,5.725262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZBH,NaN,123.01,NaN,18.010250,NaN,2.107635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZION,NaN,50.86,NaN,9.049823,NaN,1.787886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [387]:
# rv_df_t=rv_df
# rv_df=rv_df_t

In [388]:

for column in [ 'Price-to-Earnings Ratio',
        'Price-to-Book Ratio', 
       'Price-to-Sales Ratio', 'EV/EBITDA','EV/GP']:
    rv_df[column].fillna(rv_df[column].mean(),inplace=True)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
MMM,NaN,126.60,NaN,12.423945,NaN,4.969968,NaN,2.140000,NaN,8.710000,NaN,5.097105,NaN,NaN
AOS,NaN,60.03,NaN,19.490260,NaN,5.247378,NaN,3.732479,NaN,-6.222523,NaN,9.766213,NaN,NaN
ABT,NaN,113.39,NaN,21.722223,NaN,5.541762,NaN,4.470000,NaN,15.500000,NaN,8.173531,NaN,NaN
ABBV,NaN,152.83,NaN,11.042630,NaN,16.898497,NaN,4.720000,NaN,13.010000,NaN,8.471451,NaN,NaN
ACN,NaN,284.88,NaN,24.836966,NaN,7.804932,NaN,2.900000,NaN,16.970000,NaN,8.886802,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,NaN,130.33,NaN,29.221973,NaN,-4.333211,NaN,3.732479,NaN,-6.222523,NaN,9.766213,NaN,NaN
ZBRA,NaN,293.58,NaN,16.911290,NaN,5.725262,NaN,3.732479,NaN,-6.222523,NaN,9.766213,NaN,NaN
ZBH,NaN,123.01,NaN,18.010250,NaN,2.107635,NaN,3.732479,NaN,-6.222523,NaN,9.766213,NaN,NaN
ZION,NaN,50.86,NaN,9.049823,NaN,1.787886,NaN,3.732479,NaN,-6.222523,NaN,9.766213,NaN,NaN


In [389]:
rv_df.isnull().any()

Ticker                      True
Price                      False
Number of Shares to Buy     True
Price-to-Earnings Ratio    False
PE Percentile               True
Price-to-Book Ratio        False
PB Percentile               True
Price-to-Sales Ratio       False
PS Percentile               True
EV/EBITDA                  False
EV/EBITDA Percentile        True
EV/GP                      False
EV/GP Percentile            True
RV Score                    True
dtype: bool

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [390]:
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
MMM,NaN,126.60,NaN,12.423945,NaN,4.969968,NaN,2.140000,NaN,8.710000,NaN,5.097105,NaN,NaN
AOS,NaN,60.03,NaN,19.490260,NaN,5.247378,NaN,3.732479,NaN,-6.222523,NaN,9.766213,NaN,NaN
ABT,NaN,113.39,NaN,21.722223,NaN,5.541762,NaN,4.470000,NaN,15.500000,NaN,8.173531,NaN,NaN
ABBV,NaN,152.83,NaN,11.042630,NaN,16.898497,NaN,4.720000,NaN,13.010000,NaN,8.471451,NaN,NaN
ACN,NaN,284.88,NaN,24.836966,NaN,7.804932,NaN,2.900000,NaN,16.970000,NaN,8.886802,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,NaN,130.33,NaN,29.221973,NaN,-4.333211,NaN,3.732479,NaN,-6.222523,NaN,9.766213,NaN,NaN
ZBRA,NaN,293.58,NaN,16.911290,NaN,5.725262,NaN,3.732479,NaN,-6.222523,NaN,9.766213,NaN,NaN
ZBH,NaN,123.01,NaN,18.010250,NaN,2.107635,NaN,3.732479,NaN,-6.222523,NaN,9.766213,NaN,NaN
ZION,NaN,50.86,NaN,9.049823,NaN,1.787886,NaN,3.732479,NaN,-6.222523,NaN,9.766213,NaN,NaN


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [391]:

metrics={
"Price-to-Earnings Ratio":"PE Percentile",
"Price-to-Book Ratio":"PB Percentile",
"Price-to-Sales Ratio":"PS Percentile",
"EV/EBITDA":"EV/EBITDA Percentile",
"EV/GP":"EV/GP Percentile"
}
for metric in metrics.keys():
    for row in rv_df.index:
        rv_df.loc[row,metrics[metric]]=stats.percentileofscore(rv_df[metric],rv_df.loc[row,metric])/100
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
MMM,NaN,126.60,NaN,12.423945,0.246521,4.969968,0.660040,2.140000,0.177932,8.710000,0.642147,5.097105,0.095427,NaN
AOS,NaN,60.03,NaN,19.490260,0.532803,5.247378,0.697813,3.732479,0.560636,-6.222523,0.292247,9.766213,0.551690,NaN
ABT,NaN,113.39,NaN,21.722223,0.632207,5.541762,0.715706,4.470000,0.862823,15.500000,0.811133,8.173531,0.210736,NaN
ABBV,NaN,152.83,NaN,11.042630,0.196819,16.898497,0.936382,4.720000,0.874751,13.010000,0.743539,8.471451,0.220676,NaN
ACN,NaN,284.88,NaN,24.836966,0.709742,7.804932,0.809145,2.900000,0.244533,16.970000,0.842942,8.886802,0.238569,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,NaN,130.33,NaN,29.221973,0.813121,-4.333211,0.065606,3.732479,0.560636,-6.222523,0.292247,9.766213,0.551690,NaN
ZBRA,NaN,293.58,NaN,16.911290,0.431412,5.725262,0.723658,3.732479,0.560636,-6.222523,0.292247,9.766213,0.551690,NaN
ZBH,NaN,123.01,NaN,18.010250,0.461233,2.107635,0.333996,3.732479,0.560636,-6.222523,0.292247,9.766213,0.551690,NaN
ZION,NaN,50.86,NaN,9.049823,0.121272,1.787886,0.252485,3.732479,0.560636,-6.222523,0.292247,9.766213,0.551690,NaN


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [392]:
from statistics import mean
for i in rv_df.index:
    value_percentiles=[]
    for metric in metrics.keys():
        value_percentiles.append(rv_df.loc[i,metrics[metric]])
    rv_df.loc[i,"RV Score"]=mean(value_percentiles)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
MMM,NaN,126.60,NaN,12.423945,0.246521,4.969968,0.660040,2.140000,0.177932,8.710000,0.642147,5.097105,0.095427,0.364414
AOS,NaN,60.03,NaN,19.490260,0.532803,5.247378,0.697813,3.732479,0.560636,-6.222523,0.292247,9.766213,0.551690,0.527038
ABT,NaN,113.39,NaN,21.722223,0.632207,5.541762,0.715706,4.470000,0.862823,15.500000,0.811133,8.173531,0.210736,0.646521
ABBV,NaN,152.83,NaN,11.042630,0.196819,16.898497,0.936382,4.720000,0.874751,13.010000,0.743539,8.471451,0.220676,0.594433
ACN,NaN,284.88,NaN,24.836966,0.709742,7.804932,0.809145,2.900000,0.244533,16.970000,0.842942,8.886802,0.238569,0.568986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,NaN,130.33,NaN,29.221973,0.813121,-4.333211,0.065606,3.732479,0.560636,-6.222523,0.292247,9.766213,0.551690,0.456660
ZBRA,NaN,293.58,NaN,16.911290,0.431412,5.725262,0.723658,3.732479,0.560636,-6.222523,0.292247,9.766213,0.551690,0.511928
ZBH,NaN,123.01,NaN,18.010250,0.461233,2.107635,0.333996,3.732479,0.560636,-6.222523,0.292247,9.766213,0.551690,0.439960
ZION,NaN,50.86,NaN,9.049823,0.121272,1.787886,0.252485,3.732479,0.560636,-6.222523,0.292247,9.766213,0.551690,0.355666


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [393]:
rv_df.sort_values("RV Score",ascending=True,inplace=True)
rv_df=rv_df[:50]
# rv_df.reset_index(drop=True,inplace=True)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
WBA,NaN,36.33,NaN,8.073334,0.099404,1.522568,0.190855,0.240000,0.008946,-23.380000,0.005964,2.224973,0.007952,0.062624
AIG,NaN,63.78,NaN,14.204900,0.314115,1.236646,0.137177,0.890000,0.063618,-6.222523,0.292247,3.890937,0.053678,0.172167
VTRS,NaN,11.84,NaN,3.513353,0.033797,0.748514,0.085487,0.840000,0.058648,7.170000,0.614314,4.292903,0.081511,0.174751
PARA,NaN,20.64,NaN,10.530612,0.172962,0.591489,0.081511,0.440000,0.021869,5.360000,0.596421,2.498155,0.009940,0.176541
PRU,NaN,101.08,NaN,10.606506,0.176938,2.260893,0.377734,0.600000,0.033797,-6.222523,0.292247,2.171536,0.005964,0.177336
GM,NaN,36.61,NaN,5.091794,0.049702,0.785285,0.087475,0.360000,0.015905,5.670000,0.600398,7.420907,0.168986,0.184493
WRK,NaN,37.10,NaN,9.464285,0.137177,0.827774,0.091451,0.460000,0.023857,5.540000,0.598410,4.251244,0.077535,0.185686
MET,NaN,71.57,NaN,10.180654,0.165010,2.247166,0.373757,0.830000,0.055666,-6.222523,0.292247,3.717045,0.049702,0.187276
MHK,NaN,111.18,NaN,8.545733,0.113320,0.903432,0.099404,0.650000,0.036779,9.910000,0.666004,3.063830,0.027833,0.188668
MOS,NaN,47.25,NaN,4.069768,0.035785,1.389012,0.157058,0.910000,0.065606,3.270000,0.574553,6.012384,0.115308,0.189662


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [394]:
portfolio_input()

In [395]:
position_size=float(portfolio_size)/len(rv_df.index)
for i in rv_df.index:
    rv_df.loc[i,"Number of Shares to Buy"]=math.floor(position_size/rv_df.loc[i,"Price"])
rv_df

/tmp/ipykernel_850/2894507539.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_df.loc[i,"Number of Shares to Buy"]=math.floor(position_size/rv_df.loc[i,"Price"])


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
WBA,NaN,36.33,55.0,8.073334,0.099404,1.522568,0.190855,0.240000,0.008946,-23.380000,0.005964,2.224973,0.007952,0.062624
AIG,NaN,63.78,31.0,14.204900,0.314115,1.236646,0.137177,0.890000,0.063618,-6.222523,0.292247,3.890937,0.053678,0.172167
VTRS,NaN,11.84,168.0,3.513353,0.033797,0.748514,0.085487,0.840000,0.058648,7.170000,0.614314,4.292903,0.081511,0.174751
PARA,NaN,20.64,96.0,10.530612,0.172962,0.591489,0.081511,0.440000,0.021869,5.360000,0.596421,2.498155,0.009940,0.176541
PRU,NaN,101.08,19.0,10.606506,0.176938,2.260893,0.377734,0.600000,0.033797,-6.222523,0.292247,2.171536,0.005964,0.177336
GM,NaN,36.61,54.0,5.091794,0.049702,0.785285,0.087475,0.360000,0.015905,5.670000,0.600398,7.420907,0.168986,0.184493
WRK,NaN,37.10,53.0,9.464285,0.137177,0.827774,0.091451,0.460000,0.023857,5.540000,0.598410,4.251244,0.077535,0.185686
MET,NaN,71.57,27.0,10.180654,0.165010,2.247166,0.373757,0.830000,0.055666,-6.222523,0.292247,3.717045,0.049702,0.187276
MHK,NaN,111.18,17.0,8.545733,0.113320,0.903432,0.099404,0.650000,0.036779,9.910000,0.666004,3.063830,0.027833,0.188668
MOS,NaN,47.25,42.0,4.069768,0.035785,1.389012,0.157058,0.910000,0.065606,3.270000,0.574553,6.012384,0.115308,0.189662


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [396]:
writer=pd.ExcelWriter("value_strategy.xlsx",engine="xlsxwriter")
rv_df.to_excel(writer,sheet_name="Value Strategy",index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [397]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )
float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )


In [398]:
column_formats={
   'A':[ 'Ticker', string_template],
   'B':[ 'Price',dollar_template ],
   'C':[ 'Number of Shares to Buy', integer_template],
   'D':[ 'Price-to-Earnings Ratio',float_template],
   'E':[ 'PE Percentile',percent_template],
   'F':[ 'Price-to-Book Ratio',float_template],
   'G':[ 'PB Percentile',percent_template],
   'H':[ 'Price-to-Sales Ratio',float_template],
   'I':[ 'PS Percentile',percent_template],
   'J':[ 'EV/EBITDA',float_template],
   'K':[ 'EV/EBITDA Percentile',percent_template],
   'L':[ 'EV/GP',float_template],
   'M':[ 'EV/GP Percentile', percent_template],
   'N':[ 'RV Score',percent_template]
 }
for column in column_formats.keys():
    writer.sheets["Value Strategy"].set_column(f"{column}:{column}",25,column_formats[column][1])
    writer.sheets["Value Strategy"].write(f"{column}1",column_formats[column][0],column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [399]:
writer.save()